Импортируются необходимые библиотеки: pandas и numpy для работы с данными, sklearn для масштабирования данных, scipy для статистических расчетов, seaborn и matplotlib для визуализации данных.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler


Загрузка данных из CSV файла credit_train.csv с указанием разделителя ;, десятичного разделителя , и кодировки windows-1251

In [ ]:
# Загрузка данных
df = pd.read_csv('credit_train.csv', sep=';', decimal=',', encoding='windows-1251')

Удаление всех строк с пропущенными значениями из DataFrame

In [ ]:
# 1. Обработка пропусков (удаление строк с пропусками)
df = df.dropna()

Выбор только числовых столбцов из DataFrame для дальнейшей обработки.

In [ ]:
# Фильтрация числовых столбцов
numeric_df = df.select_dtypes(include=[np.number])

Расчет Z-оценок для всех числовых столбцов и удаление строк с выбросами, где абсолютное значение Z-оценок больше 3.

In [ ]:
# 2. Оценка выбросов
z_scores = np.abs(stats.zscore(numeric_df))
numeric_df_no_outliers = numeric_df[(z_scores < 3).all(axis=1)]


Вычисление корреляционной матрицы для числовых столбцов без выбросов и визуализация матрицы корреляций с помощью тепловой карты.

In [ ]:
# 3. Корреляция
correlation_matrix = numeric_df_no_outliers.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Корреляционная матрица")
plt.show()


Проведение теста Шапиро-Уилка на нормальность распределения для каждого числового столбца без выбросов. Вывод результатов теста для каждой колонки.

In [ ]:
# 4. Тест на нормальность распределения
from scipy.stats import shapiro

for column in numeric_df_no_outliers.columns:
    stat, p = shapiro(numeric_df_no_outliers[column])
    print(f'{column}: p-value={p}')
    if p > 0.05:
        print(f'Колонка {column} имеет нормальное распределение\n')
    else:
        print(f'Колонка {column} не имеет нормального распределения\n')

Масштабирование данных с использованием MinMaxScaler, приведение значений каждого столбца к диапазону от 0 до 1. Создание нового DataFrame с масштабированными данными.

In [ ]:
# 5. Масштабирование данных
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(numeric_df_no_outliers)
df_scaled = pd.DataFrame(scaled_data, columns=numeric_df_no_outliers.columns)


Сохранение преобразованных данных в новый CSV файл credit_train_processed.csv без индексации строк.

In [ ]:
# Сохранение преобразованных данных
df_scaled.to_csv('credit_train_processed.csv', index=False)


Этот код помогает подготовить данные для дальнейшего анализа или использования в моделях машинного обучения. Сначала загружаются данные из файла credit_train.csv и удаляются строки с пропущенными значениями. Затем из данных выбираются только числовые столбцы. Оцениваются и удаляются экстремальные значения (выбросы). Считается корреляция между числовыми данными, и строится тепловая карта для визуализации этих корреляций. После этого проводится тест на нормальность распределения для каждого числового столбца. Затем данные масштабируются так, чтобы все значения были в диапазоне от 0 до 1. Наконец, обработанные данные сохраняются в новый файл credit_train_processed.csv. В результате мы также получаем рисунок, отображающий корреляционную матрицу.